<p float="left">
  <img src="immo_scout24.png" width="400" />
  <img src="immo.png" width="400" />
</p>



###### <center><h1>PROJEKTARBEIT</h1></center>
### <center><h2>ANALYSE des IMMOSCOUT-DATENSATZES</h2></center>
#### <center><h3>TEIL 3 - UMGANG mit AUSREIßERN</h3></center>

**Projektarbeit-Alfatrining / Herr Axel Wemmel**

**[Nurdan Cakir]**

<ul style="list-style-type: none; padding: 0;">
  <li style="background-color:#dfa8e4; padding:10px; margin-bottom:5px; border-radius:6px;">
    <a href="#1" style="color:black; text-decoration:none;">1. Importieren der benötigten Bibliotheken</a>
  </li>
  <li style="background-color:#dfa8e4; padding:10px; margin-bottom:5px; border-radius:6px;">
    <a href="#2" style="color:black; text-decoration:none;">2. Benutzerdefinierte Funktionen zum Entfernen der Ausreißern</a>
  </li>
  <li style="background-color:#dfa8e4; padding:10px; margin-bottom:5px; border-radius:6px;">
    <a href="#3" style="color:black; text-decoration:none;">3. Untersuchungen von Ausreißern in Spalten</a>
  </li>
  <li style="background-color:#dfa8e4; padding:10px; margin-bottom:5px; border-radius:6px;">
    <a href="#4" style="color:black; text-decoration:none;">4. Fazit und Visualisierungen zur Datenanalyse</a>
  </li>
</ul>

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt
import plotly.express as px
from ipywidgets import interact, Dropdown, IntSlider, Output, widgets
# from IPython.core.display import display
from IPython.display import display
from termcolor import colored
import re

from skimpy import clean_columns
# import requests
# import geopy
# from geopy.geocoders import Nominatim

import warnings
warnings.filterwarnings("ignore")
warnings.warn("this will not show")

%matplotlib inline
# %matplotlib notebook

plt.rcParams["figure.figsize"] = (10, 6)
# plt.rcParams['figure.dpi'] = 100

sns.set_style("whitegrid")
pd.set_option('display.float_format', lambda x: '%.2f' % x)

pd.options.display.max_rows = 300
pd.options.display.max_columns = 100

In [ ]:
df0 = pd.read_csv("filling_immo_data.csv")
df = df0.copy()

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.isnull().sum()*100 / df.shape[0]

In [ ]:
df.duplicated(keep=False).sum()

In [ ]:
df = df.drop_duplicates()

In [ ]:
# Let's assign Columns to a new object named numeric_col 

numeric_col = df.select_dtypes(include="number")
display(numeric_col.columns)
numeric_col

# 2. Benutzerdefinierte Funktionen zum Entfernen der Ausreißern

In [ ]:
def first_looking(df, col):
    """
    Prints basic information about a column in a Pandas DataFrame.

    Parameters:
    -----------
    df : pandas.DataFrame
        The DataFrame to analyze.
    col : str
        The name of the column to analyze.

    Returns:
    --------
    None.

    Prints:
    -------
    column name    : str
        The name of the column being analyzed.
    per_of_nulls   : float
        The percentage of null values in the column.
    num_of_nulls   : int
        The number of null values in the column.
    num_of_uniques : int
        The number of unique values in the column.
    shape_of_df    : tuple
        The shape of the DataFrame.
    The unique values in the column and their frequency of occurrence.
    """
    print("column name    : ", col)
    print("--------------------------------")
    print("per_of_nulls   : ", "%", round(df[col].isnull().sum() * 100 / df.shape[0], 2))
    print("num_of_nulls   : ", df[col].isnull().sum())
    print("num_of_uniques : ", df[col].astype(str).nunique())
    print("shape_of_df    : ", df.shape)
    print("--------------------------------")
    print(df[col].value_counts(dropna=False))

In [ ]:
def fill(df, group_col1, group_col2, col_name, method):
    """
    Fills missing values in a column of a Pandas DataFrame `df` based on double-stage grouping and a specified filling method.

    Parameters:
    df (pandas.DataFrame): The DataFrame to operate on.
    group_col1 (str): The name of the first grouping column.
    group_col2 (str): The name of the second grouping column.
    col_name (str): The name of the column to fill missing values in.
    method (str): The filling method to use. Can be "mode", "mean", "median", "ffill", or "bfill".

    Returns:
    None.

    Prints:
    None.
    """
    
    if method == "mode":
        for group1 in list(df[group_col1].unique()):
            for group2 in list(df[group_col2].unique()):
                cond1 = df[group_col1]==group1
                cond2 = (df[group_col1]==group1) & (df[group_col2]==group2)
                mode1 = list(df[cond1][col_name].mode())
                mode2 = list(df[cond2][col_name].mode())
                if mode2 != []:
                    df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(df[cond2][col_name].mode()[0])
                elif mode1 != []:
                    df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(df[cond1][col_name].mode()[0])
                else:
                    df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(df[col_name].mode()[0])

    elif method == "mean":
        df[col_name].fillna(df.groupby([group_col1, group_col2])[col_name].transform("mean"), inplace = True)
        df[col_name].fillna(df.groupby(group_col1)[col_name].transform("mean"), inplace = True)
        df[col_name].fillna(df[col_name].mean(), inplace = True)
        
    elif method == "median":
        df[col_name].fillna(df.groupby([group_col1, group_col2])[col_name].transform("median"), inplace = True)
        df[col_name].fillna(df.groupby(group_col1)[col_name].transform("median"), inplace = True)
        df[col_name].fillna(df[col_name].median(), inplace = True)
        
    elif method == "ffill":           
        for group1 in list(df[group_col1].unique()):
            for group2 in list(df[group_col2].unique()):
                cond2 = (df[group_col1]==group1) & (df[group_col2]==group2)
                df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(method="ffill").fillna(method="bfill")
                
        for group1 in list(df[group_col1].unique()):
            cond1 = df[group_col1]==group1
            df.loc[cond1, col_name] = df.loc[cond1, col_name].fillna(method="ffill").fillna(method="bfill")            
           
        df[col_name] = df[col_name].fillna(method="ffill").fillna(method="bfill")
    
    print("COLUMN NAME    : ", col_name)
    print("--------------------------------")
    print("per_of_nulls   : ", "%", round(df[col_name].isnull().sum()*100 / df.shape[0], 2))
    print("num_of_nulls   : ", df[col_name].isnull().sum())
    print("num_of_uniques : ", df[col_name].nunique())
    print("--------------------------------")
    print(df[col_name].value_counts(dropna = False).sort_index())

In [ ]:
import plotly.express as px

def plot_categorical_distributions(df, column=None):
    """
    Eğer column parametresi verilirse sadece o sütunun dağılımını,
    verilmezse DataFrame'deki tüm kategorik sütunların dağılımını çizer.
    """
    if column:
        columns_to_plot = [column]
    else:
        columns_to_plot = df.select_dtypes(include=['object', 'category']).columns.tolist()
    
    for col in columns_to_plot:
        fig = px.histogram(
            df,
            x=col,
            title=f'Distribution of {col}',
            labels={col: col, 'count': 'Count'},
            color_discrete_sequence=['#636EFA']
        )
        fig.update_layout(xaxis_tickangle=45)
        fig.show()

In [ ]:
def remove_outliers_tukey(df, col_name):
    """
    Drops outliers from a Pandas DataFrame based on the Tukey's Fence Rule for a specific feature.
    
    Parameters:
    df (Pandas DataFrame): The input DataFrame.
    feature (str): The name of the feature to use for outlier detection.
    
    Returns:
    Pandas DataFrame: The DataFrame with the outliers removed.
    """    
    print("Number of rows before dropping outliers:", len(df))
    q1 = df[col_name].quantile(0.25)
    q3 = df[col_name].quantile(0.75)
    iqr = q3-q1  # Interquartile range
    fence_low  = q1-1.5*iqr
    fence_high = q3+1.5*iqr
    df = df.loc[(df[col_name] > fence_low) & (df[col_name] < fence_high)]
    df.reset_index(drop=True, inplace=True)
    print("Number of rows after dropping outliers:", len(df))
    return df

In [ ]:
def drop_outliers_zscore(df, feature, threshold=3):
    """
    Drops outliers from a Pandas DataFrame based on the z-score of a specific feature.
    
    Parameters:
    df (Pandas DataFrame): The input DataFrame.
    feature (str): The name of the feature to use for outlier detection.
    threshold (float, optional): The threshold value for the z-score. Defaults to 3.
    
    Returns:
    Pandas DataFrame: The DataFrame with the outliers removed.
    """
    # Print the number of rows before dropping outliers
    print("Number of rows before dropping outliers:", len(df))
    
    # Calculate the z-scores for the feature
    z_scores = np.abs((df[feature] - df[feature].mean()) / df[feature].std())

    # Drop the rows with z-scores above the threshold
    df = df[z_scores < threshold]

    # Drop the rows with z-scores above the threshold
    df.reset_index(drop=True, inplace=True)
    
    # Print the number of rows after dropping outliers
    print("Number of rows after dropping outliers:", len(df))
    
    return df.info()

# 3. Untersuchungen von Ausreißern in den Spalten

## 3.1. base_rent

In [ ]:
first_looking(df, "base_rent")

In [ ]:
class color:
    PURPLE = '\033[95m'
    CYAN = '\033[96m'
    DARKCYAN = '\033[36m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'

In [ ]:
# Let's explore Descriptive Satatistics on "price"

display(df.base_rent.describe())

# Differences between intervals
diff1 = df.base_rent.describe()['mean'] - df.base_rent.describe()['std']
diff2 = df.base_rent.describe()['25%'] - df.base_rent.describe()['min']
diff3 = df.base_rent.describe()['max'] - df.base_rent.describe()['75%']

print(f"The difference between the {color.BOLD + color.BLUE}mean value{color.BLUE + color.END} ({color.BOLD + color.GREEN}{df.base_rent.describe()['mean']:.2f}){color.BLUE + color.END} and the {color.BOLD + color.BLUE}standard deviation{color.BLUE + color.END} ({color.BOLD + color.GREEN}{df.base_rent.describe()['std']:.2f}{color.GREEN + color.END}) is {color.BOLD + color.RED}{diff1:.2f}{color.RED + color.END}.")
print(f"The difference between the {color.BOLD + color.BLUE}25th percentile{color.BLUE + color.END} ({color.BOLD + color.GREEN}{df.base_rent.describe()['25%']:.2f}){color.BLUE + color.END} and the {color.BOLD + color.BLUE}minimum value{color.BLUE + color.END} ({color.BOLD + color.GREEN}{df.base_rent.describe()['min']:.2f}{color.GREEN + color.END}) is {color.BOLD + color.RED}{diff2:.2f}{color.RED + color.END}.")
print(f"The difference between the {color.BOLD + color.BLUE}75th percentile{color.BLUE + color.END} ({color.BOLD + color.GREEN}{df.base_rent.describe()['75%']:.2f}){color.BLUE + color.END} and the {color.BOLD + color.BLUE}maximum value{color.BLUE + color.END} ({color.BOLD + color.GREEN}{df.base_rent.describe()['max']:.2f}{color.GREEN + color.END}) is {color.BOLD + color.RED}{diff3:.2f}{color.RED + color.END}.")

In [ ]:
plt.figure(figsize=(20, 6))

plt.subplot(121)
plt.hist(df.base_rent, bins=20)

plt.subplot(122)
plt.boxplot(df.base_rent, whis=3)

plt.show()

**💡Überprüfung der obersten Zeilen**

In [ ]:
df.sort_values(by=["base_rent"], ascending=False)["base_rent"].head(50)

# df.price.sort_values().head(20)

In [ ]:
df[df['base_rent'] > 10000][['base_rent', 'living_space', 'no_rooms_cleaned', 'total_rent']].describe()

In [ ]:
index_max_50_base_rent = df.sort_values(by=["base_rent"], ascending=False)["base_rent"].head(50).index
index_max_50_base_rent

In [ ]:
df.loc[[261365,   8404, 256832,  16713, 240645, 179837, 174969, 211226, 188349,
       169238, 247349, 226266, 223123, 175597, 161957,  57046, 117836,  60879,
        14113, 238788, 194351,  95802, 221105,  70814, 112005,  70865, 153484,
        38055, 146806,   1648, 259978,  65314, 263181,  63869, 170375,   9053,
       127519, 224301, 104799, 149353,  99329, 153314, 217850, 117207,  33654,
       232249,  80326, 207434, 120865,  60455]]

In [ ]:
# Berechnet den Durchschnitt der Grundmiete (base_rent) für jede Region
region_rent_mean = df.groupby('regio_1')['base_rent'].mean().sort_values(ascending=False)

# Erstellt eine Balkengrafik der durchschnittlichen Miete pro Region
plt.figure(figsize=(14, 6))
region_rent_mean.plot(kind='bar', color='skyblue')

# Titel und Achsenbeschriftungen setzen
plt.title('Durchschnittliche Grundmiete (base_rent) nach Region (Regio_1)', fontsize=14)
plt.ylabel('Durchschnittliche Miete (€)', fontsize=12)
plt.xlabel('Region (Regio_1)', fontsize=12)

# Dreht die x-Achsenbeschriftungen für bessere Lesbarkeit
plt.xticks(rotation=45, ha='right')

# Zeigt die Grafik an
plt.tight_layout()
plt.show()

In [ ]:
df[df['base_rent'] > 10000].shape[0]

In [ ]:
df = df[df['base_rent'] <= 10000]

**💡Erklärung für das Löschen der obersten 29 Mietdatensätze:** „Die obersten 29 Datensätze mit den höchsten Basis-Mietwerten wurden entfernt, da diese Werte unrealistisch hoch erscheinen und wahrscheinlich Fehler oder Ausreißer in den Daten darstellen. Solche Ausreißer können die Analyse verzerren, deshalb ist deren Entfernung wichtig, um valide und aussagekräftige Ergebnisse zu erzielen.“

In [ ]:
plt.figure(figsize=(20, 6))

plt.subplot(121)
plt.hist(df.base_rent, bins=20)

plt.subplot(122)
plt.boxplot(df.base_rent, whis=3)

plt.show()

**💡Überprüfung der niedrigsten Zeilen**

In [ ]:
df[df['base_rent'] <= 50].T

In [ ]:
df.loc[264041].description

In [ ]:
df.loc[262089].description

In [ ]:
df['base_rent'].value_counts().loc[[0, 1]]

In [ ]:
df = df[df['base_rent'] > 150.00]

**💡Erklärung für das Löschen der Datensätze mit Basis-Miete = 150:** „Datensätze mit einem Basis-Mietwert von Null, obwohl die Gesamtmiete positiv ist, wurden gelöscht. Dies deutet auf inkonsistente oder fehlerhafte Dateneinträge hin. Um eine saubere und verlässliche Datenbasis für die weitere Analyse zu gewährleisten, ist das Entfernen solcher Einträge notwendig.“

Die Verteilung der Beobachtungen im Feature „base_rent“ sieht immer noch NICHT ganz richtig aus. Versuchen wir daher, Extremwerte mit der **"Turkey's Fence Rule"** zu behandeln.

In [ ]:
df['log_base_rent'] = np.log(df['base_rent'])

In [ ]:
sns.histplot(df['log_base_rent'], kde=True)
plt.title("Log-Transformed base_rent Dağılımı")
plt.show()

In [ ]:
# Let's determine the First & Third Quantile and Inter Quantile Range 
# so we can calculate lowest and highest boundries of fence to drop extreme values

print("Number of rows before dropping outliers:", len(df))
q1 = df["log_base_rent"].quantile(0.25)
q3 = df["log_base_rent"].quantile(0.75)
iqr = q3-q1  # Interquartile range

fence_low  = q1 -3 * iqr
fence_high = q3 + 3 *iqr

df = df.loc[(df["log_base_rent"] > fence_low) & (df["log_base_rent"] < fence_high)]
df.reset_index(drop=True, inplace=True)
print("Number of rows after dropping outliers:", len(df))

In [ ]:
plt.figure(figsize=(20, 6))

plt.subplot(121)
plt.hist(df.log_base_rent, bins=20)

plt.subplot(122)
plt.boxplot(df.log_base_rent, whis=3)

plt.show()

In [ ]:
df["log_base_rent"].describe()


**💡Log-Transformation:** Da alle Werte in der Spalte base_rent positiv sind, wurde eine logarithmische Transformation durchgeführt, um die Verteilung zu normalisieren und die Auswirkung von Ausreißern zu reduzieren.
🔍 Warum eine logarithmische Transformation?
Reduktion von Schiefe (Skewness): Mietpreise sind häufig rechtsschief verteilt – das bedeutet, dass es wenige sehr hohe Werte gibt, die die Analyse verzerren können. Durch die log-Transformation wird die Verteilung symmetrischer.

Stabilisierung der Varianz: Große Werteunterschiede zwischen günstigen und teuren Wohnungen werden reduziert.

Bessere Modellierung: Viele statistische Modelle (z. B. lineare Regression) funktionieren besser, wenn die Eingabedaten annähernd normalverteilt sind.

🛠️ Was passiert technisch?
Der natürliche Logarithmus (Basis e) wird auf jeden Wert in base_rent angewendet.

Beispiel: log(1000) ≈ 6.91, log(2000) ≈ 7.60. Der Unterschied wird also komprimiert.

🔒 Wichtiger Hinweis:
Die Transformation funktioniert nur, weil alle Werte positiv sind. Wären 0 oder negative Zahlen in base_rent enthalten, würde der Logarithmus einen Fehler erzeugen.

***💡Die Verteilung der Grundrente nach der Modifikation durch Tukey's Fence Rule***

In [ ]:
from scipy import stats
plt.figure(figsize=(16, 6))

# Sample 100 observations from the 'price' column
my_data = df['log_base_rent'].sample(100)

# Calculate the mean and standard deviation of the sample
mu = np.mean(my_data)
sigma = np.std(my_data)

# Generate a normal distribution with the same mean and standard deviation as the sample
x = np.linspace(mu - 3*sigma, mu + 3*sigma, 100)
y = norm.pdf(x, mu, sigma)

# Plot the normal distribution as a dotted red line on the Q-Q plot
# plt.plot(x, y, 'r--', linewidth=2)

# Create the Q-Q plot
fig, ax = plt.subplots()
stats.probplot(my_data, dist="norm", plot=ax)

# Show the plot
plt.show()

**💡📈 Q-Q-Plot nach Log-Transformation der Miete**

Um die Verteilung der Mietpreise (base_rent) zu untersuchen, wurde eine logarithmische Transformation durchgeführt. Anschließend wurde ein Q-Q-Plot erstellt, um zu prüfen, inwieweit die transformierten Werte einer Normalverteilung folgen.

🔍 Interpretation:
Die blauen Punkte stellen die quantilenbasierten Werte der logarithmierten base_rent-Daten dar.

Die rote Linie repräsentiert eine theoretische Normalverteilung.

Je näher die Punkte an der roten Linie liegen, desto näher ist die Verteilung der Daten an der Normalverteilung.

✅ Ergebnis:
Der Q-Q-Plot zeigt, dass die log-transformierten Mietpreise annähernd normalverteilt sind, insbesondere im mittleren Bereich der Verteilung.

Leichte Abweichungen an den Rändern (Extremwerten) sind sichtbar, aber kein starkes systematisches Muster zu erkennen.

Damit eignet sich die log-transformierte Variable gut für modellbasierte Verfahren, die Normalverteilung annehmen (z. B. lineare Regression).

🧠 Fazit:
Die Log-Transformation war erfolgreich, da sie die Schiefe der ursprünglichen Verteilung reduziert hat und die transformierten Daten besser mit der Normalverteilung übereinstimmen.

## 3.2. service_charge

In [ ]:
first_looking(df, 'service_charge')

In [ ]:
df["service_charge"].describe()

In [ ]:
# Nach Region gruppieren und den Durchschnitt anzeigen
df.groupby('regio_1')['service_charge'].mean().sort_values(ascending=False)

In [ ]:
# Nach Region gruppieren und den Durchschnitt anzeigen
df.groupby('interior_qual')['service_charge'].mean().sort_values(ascending=False)

In [ ]:
df[df['service_charge'] < 40].sort_values(by='service_charge').head(20)

In [ ]:
df = df[df['service_charge'] >= 12]

**🧹 Entfernen ungewöhnlich niedriger Nebenkosten (`service_charge`)**

In Deutschland gelten sehr niedrige Betriebskosten (Nebenkosten) unterhalb von **1,20 €/m²** als unplausibel.  
Diese Einschätzung stützt sich auf:

- **Richtwerte der Jobcenter**, die meist zwischen **1,50–1,70 €/m²** liegen (je nach Region),
- **Mindestwohnflächen für Einzelpersonen** von ca. **10 m²**,
- und die allgemeine Kostenstruktur für Heizung, Müll, Hausreinigung etc.

Daraus ergibt sich eine rechnerische **absolute Untergrenze von ca. 12 €** (10 m² × 1,20 €/m²).  
Werte unter dieser Schwelle deuten häufig auf:

- fehlende Eingaben (`0`, `1`, `5` €),
- fehlerhafte Verarbeitung oder
- nicht gemeldete Nebenkosten hin.

**Maßnahme:**

Alle Einträge mit `service_charge` < **12 €** werden aus dem Datensatz entfernt,  
um Verzerrungen in der Analyse zu vermeiden.

https://www.jobcenter-mk.de/geld-zum-wohnen.html?utm_source=chatgpt.com

https://www.jobcenter-remscheid.de/kosten-der-unterkunft.html?utm_source=chatgpt.com

In [ ]:
# IQR (Interquartilsabstand) berechnen
Q1 = df['service_charge'].quantile(0.25)  # 1. Quartil (25%)
Q3 = df['service_charge'].quantile(0.75)  # 3. Quartil (75%)
IQR = Q3 - Q1  # Interquartilsabstand

print(f"Q1: {Q1}, Q3: {Q3}, IQR: {IQR}")

# Untere und obere Grenze für Ausreißer
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(f"Untere Grenze: {lower_bound}, Obere Grenze: {upper_bound}")

In [ ]:
# Anzahl der Ausreißer berechnen
outliers = df[(df['service_charge'] < lower_bound) | (df['service_charge'] > upper_bound)]
print(f"Anzahl der als Ausreißer erkannten Zeilen: {outliers.shape[0]}")

# Entfernen der Ausreißer
df_iqr_cleaned = df[(df['service_charge'] >= lower_bound) & (df['service_charge'] <= upper_bound)]
print(f"Anzahl der verbleibenden Zeilen nach dem Entfernen: {df_iqr_cleaned.shape[0]}")

In [ ]:
df = df[(df['service_charge'] >= lower_bound) & (df['service_charge'] <= upper_bound)]

In [ ]:
df['service_charge'].describe()

In [ ]:
# Plot-Größe definieren
plt.figure(figsize=(14, 6))

# Histogramm
plt.subplot(1, 2, 1)
plt.hist(df['service_charge'], bins=50, color='skyblue', edgecolor='black')
plt.title('Histogramm der Nebenkosten')
plt.xlabel('Nebenkosten (€)')
plt.ylabel('Anzahl')

# Boxplot
plt.subplot(1, 2, 2)
plt.boxplot(df['service_charge'], vert=False, patch_artist=True,
            boxprops=dict(facecolor='lightgreen', color='black'),
            medianprops=dict(color='black'),
            whiskerprops=dict(color='black'),
            capprops=dict(color='black'),
            flierprops=dict(markerfacecolor='gray', marker='D'))

plt.title('Boxplot der Nebenkosten')
plt.xlabel('Nebenkosten (€)')

plt.tight_layout()
plt.show()


In [ ]:
df.shape

## 3.3. heating_costs

In [ ]:
first_looking(df, 'heating_costs')

In [ ]:
df['heating_costs'] = df['heating_costs'].replace(0, np.nan)

In [ ]:
df['heating_costs'] = df['heating_costs'].fillna(
    df.groupby(['regio_1', 'heating_type_cleaned'])['heating_costs'].transform('median')
)

In [ ]:
print("NaN        :", df['heating_costs'].isna().sum())
print("0          :", (df['heating_costs'] == 0).sum())
print("nunique :", df['heating_costs'].nunique())
print("shape :", df.shape)

In [ ]:
plt.figure(figsize=(8,6))
plt.boxplot(df['heating_costs'], vert=False)
plt.title('Heating Costs Boxplot')
plt.xlabel('Heating Costs')
plt.show()

In [ ]:
max_idx = df['heating_costs'].idxmax()
print(df.loc[max_idx])

In [ ]:
df[df['heating_costs'] > 1800.00][['heating_costs', 'base_rent', 'service_charge', 'total_rent']]                                                   

In [ ]:
to_drop = df[df['heating_costs'] > 1800].index
df = df.drop(to_drop)

In [ ]:
df['heating_costs'].describe()

In [ ]:
df.shape

## 3.4. total_rent

In [ ]:
first_looking(df, 'total_rent')

In [ ]:
df['total_rent'].describe()

In [ ]:
df[df['total_rent'] == 0][['total_rent', 'base_rent', 'service_charge', 'heating_costs']]

In [ ]:
mask = df['total_rent'] == 0

In [ ]:
df.loc[mask, 'total_rent'] = (
    df.loc[mask, 'base_rent'] +
    df.loc[mask, 'service_charge'] +
    df.loc[mask, 'heating_costs']
)

In [ ]:
df[df['total_rent'] > 10000][['total_rent', 'base_rent', 'service_charge', 'heating_costs']]

In [ ]:
high_mask = df['total_rent'] > 10000

In [ ]:
df.loc[high_mask, 'total_rent'] = (
    df.loc[high_mask, 'base_rent'] +
    df.loc[high_mask, 'service_charge'] +
    df.loc[high_mask, 'heating_costs']
)

In [ ]:
# Histogram
plt.subplot(1, 2, 1)
sns.histplot(df['total_rent'], bins=50, kde=True)
plt.title('Histogram of Total Rent')
plt.xlabel('Total Rent')
plt.ylabel('Frequency')

# Boxplot
plt.subplot(1, 2, 2)
sns.boxplot(x=df['total_rent'])
plt.title('Boxplot of Total Rent')

plt.tight_layout()
plt.show()

In [ ]:
df[df['total_rent'] > 4000][['base_rent', 'regio_1', 'year_category', 'no_rooms_cleaned', 'living_space', 'type_of_flat']]

In [ ]:
(df['total_rent'] > 4000).sum()

In [ ]:
df = df[df['total_rent'] <= 4000]

In [ ]:
# Histogram
plt.subplot(1, 2, 1)
sns.histplot(df['total_rent'], bins=50, kde=True)
plt.title('Histogram of Total Rent')
plt.xlabel('Total Rent')
plt.ylabel('Frequency')

# Boxplot
plt.subplot(1, 2, 2)
sns.boxplot(x=df['total_rent'])
plt.title('Boxplot of Total Rent')

plt.tight_layout()
plt.show()

## 3.5. no_rooms_cleaned

In [ ]:
first_looking(df, 'no_rooms_cleaned')

In [ ]:
df['no_rooms_cleaned'].describe()

In [ ]:
# Histogram
plt.subplot(1, 2, 1)
sns.histplot(df['no_rooms_cleaned'], bins=5, kde=True)
plt.title('Histogram of Number_Rooms')
plt.xlabel('no_rooms')
plt.ylabel('Frequency')

# Boxplot
plt.subplot(1, 2, 2)
sns.boxplot(x=df['no_rooms_cleaned'])
plt.title('Boxplot of Number_Rooms')

plt.tight_layout()
plt.show()

In [ ]:
df = df[df['no_rooms_cleaned'] <= 8]

In [ ]:
df.shape

## 3.6. floor

In [ ]:
first_looking(df, 'floor')

**💡Erklärung zur Datenbereinigung der 'floor'-Spalte:**

In unseren Daten finden sich einige außergewöhnliche Werte bei der Anzahl der Stockwerke, wie z.B. 99, 999 oder sogar 650. Zur Einordnung: Das höchste Gebäude z.B. in Berlin ist etwa 176 Meter hoch, was ungefähr 48 Stockwerken(als Büroanlage) entspricht. Werte, die deutlich darüber liegen, sind daher als Ausreißer zu betrachten und entsprechen vermutlich fehlerhaften oder kodierten Daten.

Aus diesem Grund werden wir bei der Datenbereinigung nur Stockwerkangaben bis maximal 48 berücksichtigen und alle darüber hinausgehenden Werte als Ausreißer entfernen oder als fehlend kennzeichnen. So stellen wir sicher, dass unsere Analyse auf realistischen und verlässlichen Daten basiert.

https://www.rbb24.de/panorama/beitrag/av24/berlin-hochhaus-architektur-stadtbild-wolkenkratzer-tower-.html

https://reise-nach-leipzig.de/sehenswuerdigkeiten-leipzig/hochhaeuser-leipzig/#:~:text=Das%20City%20Hochhaus%20ist%20das%20zweith%C3%B6chste%20Geb%C3%A4ude%20Ostdeutschlands,Panorama-Tower%20auch%20schon%20mal%20als%20Leipziger%20Weisheitszahn%20bezeichnet.**

In [ ]:
df = df[df['floor'] <= 16]

In [ ]:
df['floor'].value_counts()

## 3.7. thermal_char

In [ ]:
first_looking(df, 'thermal_char')

In [ ]:
df['thermal_char'].describe()

In [ ]:
# Histogram
plt.subplot(1, 2, 1)
sns.histplot(df['thermal_char'], bins=50, kde=True)
plt.title('Histogram of thermal_char')
plt.xlabel('thermal_char')
plt.ylabel('Frequency')

# Boxplot
plt.subplot(1, 2, 2)
sns.boxplot(x=df['thermal_char'])
plt.title('Boxplot of thermal_char')

plt.tight_layout()
plt.show()

In [ ]:
df[df['thermal_char'] > 300][['thermal_char', 'energy_efficiency_class', 'regio_1', 'living_space']]

In [ ]:
df[df['thermal_char'] > 300][['living_space']].mean()

In [ ]:
df = df[df['thermal_char'] <= 300]

In [ ]:
df.shape

## 3.8. no_park_spaces

In [ ]:
first_looking(df, 'no_park_spaces')

In [ ]:
df['no_park_spaces'].describe()

In [ ]:
df = df[df['no_park_spaces'] <= 10]

In [ ]:
df['no_park_spaces_cleaned'] = df['no_park_spaces'].apply(lambda x: x if x <= 5 else 6)

In [ ]:
df['no_park_spaces_cleaned'].value_counts()

In [ ]:
df.drop(columns =['no_park_spaces'], inplace=True)

## 3.9. living_space

In [ ]:
first_looking(df, 'living_space')

In [ ]:
df['living_space'].describe()

In [ ]:
# Histogram
plt.subplot(1, 2, 1)
sns.histplot(df['living_space'], bins=100, kde=True)
plt.title('Histogram of living_space')
plt.xlabel('living_space')
plt.ylabel('Frequency')

# Boxplot
plt.subplot(1, 2, 2)
sns.boxplot(x=df['living_space'])
plt.title('Boxplot of living_space')

plt.tight_layout()
plt.show()

In [ ]:
df[df['living_space'] > 200][['total_rent', 'living_space', 'type_of_flat', 'floor', 'garden']]

In [ ]:
df[df['living_space'] < 12][['total_rent', 'living_space', 'type_of_flat', 'floor', 'garden']]

In [ ]:
df = df[(df['living_space'] >= 12) & (df['living_space'] <= 200)]

# 4. Fazit und Visualisierungen zur Datenanalyse

Nach der gründlichen Bereinigung der Ausreißer (z. B. bei living_space, base_rent, service_charge, no_park_spaces) wurde das Datenset von ursprünglich 268.850 Einträgen auf 248.397 Einträge reduziert. Damit konnten extreme Werte, die auf fehlerhafte oder seltene Eingaben hinweisen, entfernt werden – ohne dabei wichtige Informationen zu verlieren.

Parallel dazu wurden auch visuelle Analysen durchgeführt, um die Verteilungen numerischer und kategorialer Merkmale besser zu verstehen. Mithilfe von Histogrammen, Boxplots und gruppierten Balkendiagrammen konnten unter anderem folgende Beobachtungen gemacht werden:

✔️ Einige Merkmale (z. B. base_rent, total_rent, living_space) zeigten rechtssteile Verteilungen, was auf Ausreißer oder lange "Preisschwänze" hinwies.

✔️ Kategoriale Merkmale wie region, year_category oder interior_qual zeigten teils deutliche Unterschiede im durchschnittlichen Mietpreis.

✔️ Die Transformation der Zielvariable in log_base_rent sorgte für eine stabilere Verteilung und ist somit besser für das Modellieren geeignet.

Die bereinigten Daten sind nun deutlich konsistenter und besser geeignet für die nächste Phase der Analyse, insbesondere für das Feature Engineering und die Modellierung.

In [ ]:
df.head(3).T

In [ ]:
numeric_col = df.select_dtypes(include='number')
numeric_col.head()

In [ ]:
plt.figure(figsize=(10, 8))

sns.heatmap(numeric_col.corr(), annot=True, cmap="Blues", linewidths=0.2, annot_kws={"size": 12});

In [ ]:
from termcolor import cprint

def multicolinearity_control(df):                    
    df_temp = df.corr()
    count = 'Done'
    feature =[]
    collinear= []
    for col in df_temp.columns:
        for i in df_temp.index:
            if abs(df_temp[col][i] > .6 and df_temp[col][i] < 1):
                    feature.append(col)
                    collinear.append(i)
                    cprint(f"Multicolinearity alert in between --> {col} - {i} --> {round(df_temp[col][i], 6)}", "red", attrs=["bold"])
    else:
#         cprint(f"There is NO multicollinearity problem.", "blue", attrs=["bold"])
        pass

In [ ]:
multicolinearity_control(numeric_col)

In [ ]:
target = 'log_base_rent'

corr_by_target = numeric_col.corr()[target].sort_values()
corr_by_target

In [ ]:
ax = sns.barplot(y = corr_by_target.index, x = corr_by_target)
plt.xticks(rotation=90)
plt.tight_layout()

for container in ax.containers:
    ax.bar_label(container, fontsize=11, rotation=0, label_type='center', color="white")

In [ ]:
df.select_dtypes(include=['object', 'category']).columns.tolist()

In [ ]:
categorical_columns = [
    'regio_1', 
    'telekom_tv_offer', 
    'interior_qual', 
    'pets_allowed',
    'type_of_flat', 
    'heating_type_cleaned', 
    'upload_speed_category', 
    'condition_grouped', 
    'year_category',
    'refurbish_category',
    'firing_types_simplified'
]

for col in categorical_columns:
    # Ortalama base_rent değerlerine göre sıralı dataframe
    df_grouped = df.groupby(col, as_index=False)['base_rent'].mean().sort_values('base_rent')
    
    fig = px.bar(
        df_grouped,
        x=col,
        y='base_rent',
        title=f'Durchschnittliche base_rent nach {col}',
        color='base_rent',
        color_continuous_scale='Viridis',
    )
    fig.update_layout(xaxis_tickangle=45)
    fig.show()

In [ ]:
count_regio = df.value_counts('regio_1').reset_index()
plt.pie(count_regio['count'], labels=None, startangle=90, wedgeprops={'linewidth': 1, 'edgecolor': 'white'})
plt.legend(count_regio['regio_1'],title="Regions", loc="center left", bbox_to_anchor=(1, 0.5), fontsize=10);

In [ ]:
boolean_columns = ['balcony', 'cellar', 'lift', 'has_kitchen', 'garden', 'newly_const']
mean_rent_df = pd.DataFrame()

for col in boolean_columns:
    grouped_means = df[[col,'base_rent']].groupby(col)['base_rent'].mean().reset_index()
    grouped_means['Column'] = col
    grouped_means.columns = ['Value','Mean Base Rent','Feature']
    mean_rent_df = pd.concat([mean_rent_df,grouped_means],axis=0)

mean_rent_df = mean_rent_df[['Feature','Value','Mean Base Rent']]
mean_rent_df

In [ ]:
mean_charge_df = pd.DataFrame()
for col in boolean_columns:
    grouped_means = df[[col,'service_charge']].groupby(col)['service_charge'].mean().reset_index()
    grouped_means['Column'] = col
    grouped_means.columns = ['Value','Mean Service Charge','Feature']
    mean_charge_df = pd.concat([mean_charge_df,grouped_means],axis=0)

mean_charge_df = mean_charge_df[['Feature','Value','Mean Service Charge']]
mean_charge_df

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

sns.barplot(data=mean_rent_df,x='Feature',y='Mean Base Rent',hue='Value', ax=axes[0])

sns.barplot(data=mean_charge_df,x='Feature',y='Mean Service Charge',hue='Value', ax=axes[1])

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(14, 9))

sns.barplot(data=df, x='no_rooms_cleaned', y='total_rent', ax=axes[1, 0],
            palette='viridis')

sns.barplot(data=df, x='floor', y='total_rent', ax=axes[1, 1],
            palette='plasma')

sns.barplot(data=df, x='type_of_flat', y='total_rent', ax=axes[0, 0],
            palette='Set2')
axes[0, 0].set_xticklabels(axes[0, 0].get_xticklabels(), rotation=90)

sns.barplot(data=df, x='heating_type_cleaned', y='total_rent', ax=axes[0, 1],
            palette='Set3')
axes[0, 1].set_xticklabels(axes[0, 1].get_xticklabels(), rotation=90)

sns.barplot(data=df, x='condition_grouped', y='total_rent', ax=axes[0, 2],
            palette='pastel')
axes[0, 2].set_xticklabels(axes[0, 2].get_xticklabels(), rotation=90)

plt.suptitle('Durchschnittliche Gesamtmiete im Vergleich verschiedener Merkmale', fontsize=16)
plt.tight_layout()
plt.show()


In [ ]:
df.info()

In [ ]:
df.duplicated(keep=False).sum()

In [ ]:
df.to_csv('full_immo_data.csv', index=False)

<a id="6"></a>
 

## <p style="background-color:#FDFEFE; font-family:newtimeroman; color:#9d4f8c; font-size:150%; text-align:center; border-radius:10px 10px;">Das Ende der Ausreißerbehandlung (Teil 03)</p>